In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Human Image Detection Eval**

In [2]:
!git clone https://github.com/semajyllek/ioqm.git

Cloning into 'ioqm'...
remote: Enumerating objects: 63, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (43/43), done.
remote: Total 63 (delta 32), reused 47 (delta 16), pack-reused 0
Receiving objects: 100% (63/63), 13.40 KiB | 2.68 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [3]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.3/20.3 MB 92.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.8/298.8 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 7.2 MB/s eta 0:

In [4]:
%cd ioqm

/content/ioqm


In [5]:
from typing import List
from utils import parse_objects_and_quantities
from pathlib import Path
import gradio as gr
import random
import json
import os

In [6]:
def get_random_image(img_path, max_images=16175):
  rand_n = random.sample(range(1, max_images), 1)[0]
  rand_img = img_path / os.listdir(img_path)[rand_n]
  return rand_img

In [7]:
def get_expected_labels(img_path) -> List[str]:
  expected_obs = parse_objects_and_quantities(img_path.stem.replace('_', ' '))
  return list(expected_obs.keys())

def eval_interface(img_path: Path, score_path: Path):
  expected_labels = get_expected_labels(img_path)

  def process_counts(*counts):
    count_dict = {}
    for labl, cnt in zip(expected_labels, counts):
      count_dict[labl] = cnt

    with open(score_path, 'a') as f:
      json.dump({img_path.name: count_dict}, f)
      f.write('\n')
    demo.close()


  with gr.Blocks() as demo:
    gr.Image(img_path)
    counts = []
    for labl in expected_labels:
      counts.append(gr.Number(label=labl))

    submit_btn = gr.Button("submit counts")
    submit_btn.click(fn=process_counts, inputs=counts, outputs=None)

  demo.launch()


In [63]:
sd_v1_5_img_folder = Path("/content/drive/MyDrive/stable-diffusion-v1-5_images")
sd_v1_5_img_v2_folder = Path("/content/drive/MyDrive/mini_prompts_v2_stable-diffusion-v1-5_images")
sd_xl_refiner_1_img_folder = Path("/content/drive/MyDrive/stable-diffusion-xl-refiner-1.0_images")
sd_xl_refiner_1_img_v2_folder = Path("/content/drive/MyDrive/mini_prompts_v2_stable-diffusion-xl-refiner-1.0_images")

img_folder = sd_v1_5_img_v2_folder
human_scores_path = img_folder.parent / f"{img_folder.name}_human_scores"
max_images = len(os.listdir(img_folder))

In [66]:
rand_img_path = get_random_image(sd_xl_refiner_1_img_v2_folder, max_images=max_images)
eval_interface(rand_img_path, human_scores_path)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8957f0fd25ab20f5a2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
